In [4]:
import pandas as pd
from datetime import datetime

def get_worldclim_file_names(movement_datatset_file_path, worldclim_file_name_prefix):
    df = pd.read_csv(movement_datatset_file_path, delimiter=',', low_memory=False)
    df['timestamp_datetime'] = pd.to_datetime(df['timestamp'])
    
    start_date = datetime(2010, 1, 1, 0, 0)
    end_date = datetime(2018, 12, 31, 23, 59)
    df = df.loc[(df['timestamp_datetime'] >= start_date) & (df['timestamp_datetime'] <= end_date)]
    
    dates = df['timestamp_datetime'].dt.date.to_list()
    dates = list(set(dates)) # remove duplicate dates

    result = []
    for date in dates:
        year_month = date.strftime("%Y-%m")
        worldclim_file_name = worldclim_file_name_prefix + year_month
        result.append(worldclim_file_name)
        
    return result

In [13]:
import gdal

def convert_tif_to_asc(source_folder, destination_folder, file_names):
    for file in file_names:
        source = '../WorldClimDatasets/' + source_folder + '/' + file + '.tif'
        destination = destination_folder + '/' + file + '.asc'
        
        print(source + ' -> ' + destination)
        
        # convert .tif to .asc
        ds_tif = gdal.Open(source)
        ds_asc = gdal.Translate(destination, ds_tif)
        
        # replace nan values
        with open(destination, 'r') as file:
            data = file.read()
            data = data.replace('nan', '-9999')
        with open(source, 'w') as file:
            file.write(data)

In [15]:
import os

movement_datasets_info = [
    '../MovebankDatasets/e-ObsGPRSHimalayanGriffon-Bhutan-MPIAB',
    '../MovebankDatasets/EurasianGriffonVultures1HzHUJ(Israel)',
    '../MovebankDatasets/Griffonvulture[fdlmes.gr]',
    '../MovebankDatasets/GriffonVultureAlbstadtSalzburg(Gypsi)',
    '../MovebankDatasets/High-altitudeflightsofHimalayanvultures(datafromSherubetal.2016)',
    '../MovebankDatasets/LifeTrackGriffonVultureCroatia',
    '../MovebankDatasets/Long-rangeadultmovementsof3vulturespecies(datafromSpiegeletal.2015)',
    '../MovebankDatasets/RaptorsNABUMoessingenpublic',
    '../MovebankDatasets/SoaringflightinEurasiangriffonvultures(HUJ)(datafromHarelandNathan,2018)'
]

weather_datasets_info = [
    # folder name, file name prefix
    ('wc2.1_2.5m_prec_2010-2018', 'wc2.1_2.5m_prec_'),
    ('wc2.1_2.5m_tmax_2010-2018', 'wc2.1_2.5m_tmax_')
]
destination_folder = '../FinalDatasets/Weather'


for weather_dataset_info in weather_datasets_info:    
    final_worldclim_file_names = []

    # filter files by dates from movement datasets
    for movement_dataset_file_path in movement_datasets_info:
        movement_dataset_files = os.listdir(movement_dataset_file_path)
        for file in movement_dataset_files:       
            worldclim_file_names = get_worldclim_file_names(movement_dataset_file_path + '/' + file, weather_dataset_info[1])
            final_worldclim_file_names = final_worldclim_file_names + worldclim_file_names

    # remove duplicate files
    final_worldclim_file_names = list(set(final_worldclim_file_names))
    final_worldclim_file_names.sort()
    
    # convert files to .asc
    convert_tif_to_asc(weather_dataset_info[0], destination_folder, final_worldclim_file_names)

../WorldClimDatasets/wc2.1_2.5m_prec_2010-2018/wc2.1_2.5m_prec_2010-11.tif
../FinalDatasets/Weather/wc2.1_2.5m_prec_2010-11.asc


KeyboardInterrupt: 